<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Before-you-start" data-toc-modified-id="Before-you-start-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Before you start</a></span></li><li><span><a href="#Authentication-setup" data-toc-modified-id="Authentication-setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Authentication setup</a></span></li><li><span><a href="#Hands-off-workflow" data-toc-modified-id="Hands-off-workflow-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Hands-off workflow</a></span></li></ul></div>

# Access Sentinel-6 NRT Data

This notebook shows a simple way to maintain a local time series of [Sentinel-6](#) NRT data using the [CMR Search API](#). It downloads granules the ingested since the previous run to a designated data folder and overwrites a hidden file inside with the timestamp of the CMR Search request on success.

> User note: The notebook actually points to a MODIS SST collection for now ([10.5067/GHMDA-2PJ19](https://doi.org/10.5067/GHMDA-2PJ19). It'll work just the same for Sentinel-6.

## Before you start

Before you beginning this tutorial, make sure you have an Earthdata account [https://uat.urs.earthdata.nasa.gov](https://uat.urs.earthdata.nasa.gov).

Accounts are free to create and take just a moment to set up.

## Authentication setup

*You'll probably need to use the netrc method when running from command line.* 

We need some boilerplate up front to log in to Earthdata Login.  The function below will allow Python
scripts to log into any Earthdata Login application programmatically.  To avoid being prompted for
credentials every time you run and also allow clients such as curl to log in, you can add the following
to a `.netrc` (`_netrc` on Windows) file in your home directory:

```
machine uat.urs.earthdata.nasa.gov
    login <your username>
    password <your password>
```

Make sure that this file is only readable by the current user or you will receive an error stating
"netrc access too permissive."

`$ chmod 0600 ~/.netrc` 


In [1]:
from urllib import request
from http.cookiejar import CookieJar
import getpass
import netrc

def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.

    Valid endpoints include:
        uat.urs.earthdata.nasa.gov - Earthdata Login UAT (Harmony's current default)
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('Please provide your Earthdata Login credentials to allow data access')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (endpoint))
        username = input('Username:')
        password = getpass.getpass()

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)

In [2]:
setup_earthdata_login_auth('uat.urs.earthdata.nasa.gov')

## Hands-off workflow

This first cell accepts parameters from [`papermill`](https://papermill.readthedocs.io) whenever this notebook is executed from the command line. It functions like any other notebook when run in the notebook server.

Learn  how to run the notebook from the command line at this link:    
https://papermill.readthedocs.io/en/latest/usage-execute.html#execute-a-notebook-with-parameters

The variables below (`mins`, `cmr`, `ccid`, and `data`) determine the notebook defaults for when it's run from the command line with papermill.

In [3]:
mins = 20  # how far back to search for granules, in minutes?

cmr = "cmr.uat.earthdata.nasa.gov"  # CMR, UAT domain (cmr.earthdata.nasa.gov)

ccid = "C1234724470-POCLOUD"  # CMR concept-id of the desired collection

data = "resources/nrt"  # path to the local NRT data directory

**In this initial example, I pointed the Python variable `data` at a test directory inside the *resources* folder adjacent to this notebook ([`resources/nrt`](resources/nrt/)).** That test data folder is `.gitignore`d. Because it doesn't get pushed to GitHub, this notebook makes it during the first run after the repo is cloned, assuming the user hasn't provided a local directory in the cell above or as an argument from the command line.

In [4]:
from os import makedirs
from os.path import isdir, basename
from urllib.request import urlopen, urlretrieve
from datetime import datetime, timedelta
from json import dumps, loads

**The search retrieves granules ingested during the last `n` minutes.** A file in your local data dir  file that tracks updates to your data directory, if one file exists. The CMR Search falls back on the ten minute window if not.

In [5]:
timestamp = (datetime.utcnow()-timedelta(minutes=mins)).strftime("%Y-%m-%dT%H:%M:%SZ")
timestamp

'2020-07-27T21:56:41Z'

**How to use the notebook:**

Run it to update your local copy of NRT data as needed. There are at least a few ways to do that:

* from the notebook server, 
* from the command line with papermill as documented above, and
* with a job scheduler like cron for routine updates.

It overwrites a hidden file called `.update` with a timestamp after each successful run. That file lives hidden in your data directory. You can modify the snippet below if that requirement won't work for you. Saving a timestamp to a file isn't a requirement to do this.

In [6]:
if not isdir(data):
    print(f"NOTE: Making new data directory at '{data}'. (This is the first run.)")
    makedirs(data)
else:
    try:
        with open(f"{data}/.update", "r") as f:
            timestamp = r.read()
    except FileNotFoundError:
        print("WARN: No .update in the data directory. (Is this the first run?)")
    else:
        print(f"NOTE: .update found in the data directory. (The last run was at {timestamp}.)")

WARN: No .update in the data directory. (Is this the first run?)


There are several ways to query for CMR updates that occured during a given timeframe. Read on in the CMR Search documentation:

* https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#c-with-new-granules (Collections)
* https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#c-with-revised-granules (Collections)
* https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#g-production-date (Granules)
* https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#g-created-at (Granules)

The `created_at` parameter works for our purposes. It's a granule search parameter that returns the records ingested since the input timestamp.

In [7]:
params = {
    'scroll': "true",
    'page_size': 2000,
    'sort_key': "-start_date",
    'collection_concept_id': ccid, 
    'created_at': timestamp,
    # Limit results to coverage for .5deg bbox in Gulf of Alaska:
    'bounding_box': "-146.5,57.5,-146,58",
}

params

{'scroll': 'true',
 'page_size': 2000,
 'sort_key': '-start_date',
 'collection_concept_id': 'C1234724470-POCLOUD',
 'created_at': '2020-07-27T21:56:41Z',
 'bounding_box': '-146.5,57.5,-146,58'}

Join the parameters dictionary into the query string by joining the parameters and values with `=`, then the `parameter=value` pairs to each other with `&`.

In [8]:
query = "&".join([f"{p}={v}" for p,v in params.items()])
query

'scroll=true&page_size=2000&sort_key=-start_date&collection_concept_id=C1234724470-POCLOUD&created_at=2020-07-27T21:56:41Z&bounding_box=-146.5,57.5,-146,58'

Append to the CMR Search endpoint for collections.

In [9]:
url = f"https://{cmr}/search/granules.umm_json?{query}"
print(url)

https://cmr.uat.earthdata.nasa.gov/search/granules.umm_json?scroll=true&page_size=2000&sort_key=-start_date&collection_concept_id=C1234724470-POCLOUD&created_at=2020-07-27T21:56:41Z&bounding_box=-146.5,57.5,-146,58


Download the granule records that match our search parameters.

In [10]:
with urlopen(url) as f:
    results = loads(f.read().decode())

print(f"{results['hits']} new granules ingested for '{ccid}' since '{timestamp}'.")

2 new granules ingested for 'C1234724470-POCLOUD' since '2020-07-27T21:56:41Z'.


Neatly print the first granule's record for reference (assuming at least one was returned).

In [11]:
if len(results['items'])>0:
    print(dumps(results['items'][0], indent=2))

{
  "meta": {
    "concept-type": "granule",
    "concept-id": "G1236615656-POCLOUD",
    "revision-id": 1,
    "native-id": "20020710140505-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0",
    "provider-id": "POCLOUD",
    "format": "application/vnd.nasa.cmr.umm+json",
    "revision-date": "2020-07-27T22:00:58Z"
  },
  "umm": {
    "RelatedUrls": [
      {
        "URL": "https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20020710140505-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc",
        "Type": "GET DATA",
        "Description": "The base directory location for the granule."
      },
      {
        "URL": "https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-public/MODIS_A-JPL-L2P-v2019.0/20020710140505-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.cmr.json",
        "Type": "EXTENDED METADATA",
        "Description": "File to download"
      },
      {
        "URL": "https://archive.podaac.uat.earthdata.nasa.gov/s3c

The link for http access denoted by `"Type": "GET DATA"` in the list of `RelatedUrls`.

Grab the download URL, but do it in a way that'll work for search results returning any number of granule records:

In [12]:
downloads = [r['umm']['RelatedUrls'][0]['URL'] for r in results['items']]
downloads

['https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20020710140505-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc',
 'https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20020704130506-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc']

Finish by downloading the files to the data directory in a loop. Overwrite `.update` with a new timestamp on success.

*USER NOTE: This cell only downloads 10 files for now. Uncomment line 1, delete line 2 to download more than 10.*

In [13]:
#for f in downloads:
for f in downloads[:10]:
    try:
        urlretrieve(f, f"{data}/{basename(f)}")
    except Exception as e:
        print(f"[{datetime.now()}] FAILURE: {f}\n\n{e}\n")
        raise e
    else:
        print(f"[{datetime.now()}] SUCCESS: {f}")

[2020-07-27 18:17:05.376730] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20020710140505-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-07-27 18:17:12.642735] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20020704130506-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc


If there were updates to the local time series during this run and no exceptions were raised during the download loop, then overwrite the timestamp file that tracks updates to the data folder (`resources/nrt/.update`):

In [14]:
if len(results['items'])>0:
    with open(f"{data}/.update", "w") as f:
        f.write(timestamp)